# Auto Registration

This is a *OpenCV* and *Gdal* for autoregistration of 2 images

---

1. Load Images and Libraries

In [1]:
import cv2
import os
from osgeo import gdal, osr
import rasterio
import numpy as np

os.chdir(r"D:\1. PROJECT\satellite_etl\satellite\test")
os.getcwd()

'D:\\1. PROJECT\\satellite_etl\\satellite\\test'

1.1 Checks raw bit size 8

In [4]:
# Open file and inspect
with rasterio.open("sss.tif") as src:
    raw = src.read(1)  # read first band
    print("Raw image shape:", raw.shape, "dtype:", raw.dtype)

with rasterio.open("oldimage.tif") as src:
    ref = src.read(1)
    print("Ref image shape:", ref.shape, "dtype:", ref.dtype)
  
# Converts 16bit to 8bits by normalization  
raw = cv2.convertScaleAbs(raw, alpha=(255.0/raw.max()))
ref = cv2.convertScaleAbs(ref, alpha=(255.0/ref.max()))

Raw image shape: (2309, 2018) dtype: uint16
Ref image shape: (2809, 2550) dtype: uint16


In [ ]:
raw = cv2.imread("sss.tif", cv2.IMREAD_GRAYSCALE)
ref = cv2.imread("oldimage.tif", cv2.IMREAD_GRAYSCALE)



❌ raw image (sss.tif) not found or unreadable
❌ ref image (oldimage.tif) not found or unreadable


2. Detect Features and Match

In [5]:
# ORB detector (tuned)
orb = cv2.ORB_create(10000, edgeThreshold=15, patchSize=31)

# Detect + compute descriptors
kp1, des1 = orb.detectAndCompute(raw, None)
kp2, des2 = orb.detectAndCompute(ref, None)

# Brute-force matcher with KNN
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
matches = bf.knnMatch(des1, des2, k=2)

# Lowe's ratio test
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append(m)

print("Good matches:", len(good))

# Need at least 4 points for homography
if len(good) >= 4:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    # Homography with RANSAC
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # Warp raw image to reference
    h, w = ref.shape
    aligned = cv2.warpPerspective(raw, H, (w, h))
    cv2.imwrite("aligned_opencv.tif", aligned)
    print("✅ Alignment done, saved as aligned_opencv.tif")

    # Debug: draw matches
    debug_matches = cv2.drawMatches(raw, kp1, ref, kp2, good[:50], None,
                                    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.imwrite("debug_matches.png", debug_matches)
    print("🔍 Saved debug_matches.png for inspection")
else:
    print("❌ Not enough good matches for homography")


Good matches: 28
✅ Alignment done, saved as aligned_opencv.tif
🔍 Saved debug_matches.png for inspection


In [6]:
matched_img = cv2.drawMatches(raw, kp1, ref, kp2, good[:50], None, flags=2)
cv2.imwrite("matches_preview.jpg", matched_img)

True

3. Estimate Transformation

4. Pass Alignment into GDAL (Georeference)

In [7]:
# Use the filename, not the array
ref_path = "oldimage.tif"
aligned_path = "aligned_opencv.tif"

ref_ds = gdal.Open(ref_path)
proj = ref_ds.GetProjection()
geotransform = ref_ds.GetGeoTransform()

# Open the aligned image (the one from warpPerspective)
aligned_ds = gdal.Open(aligned_path)

# Save aligned image with reference geoinfo
driver = gdal.GetDriverByName("GTiff")
out_ds = driver.CreateCopy("georeferenced.tif", aligned_ds)

out_ds.SetProjection(proj)           # assign CRS
out_ds.SetGeoTransform(geotransform) # assign geotransform
out_ds.FlushCache()

print("✅ Saved georeferenced.tif with spatial reference from oldimage.tif")


✅ Saved georeferenced.tif with spatial reference from oldimage.tif


c:\Users\deleo\miniconda3\envs\webgisbackend\Lib\site-packages\osgeo\gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [ ]:
import os
import cv2
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import Affine
from osgeo import gdal

# ---------- User settings ----------
WORKDIR = r"D:\1. PROJECT\satellite_etl\satellite\test"
RAW_PATH = "sss.tif"        # source to be aligned (may be multi-band, e.g., 16-bit)
REF_PATH = "oldimage.tif"   # reference with correct geo-transform & CRS
OUT_PATH = "georeferenced.tif"
DEBUG_MATCHES = "debug_matches.png"
ALIGNED_PREVIEW = "aligned_opencv_preview.tif"  # single-band preview
MIN_MATCH_COUNT = 4
RATIO_TEST = 0.75
# Choose interpolation: cv2.INTER_LINEAR (continuous) or cv2.INTER_NEAREST (categorical)
INTERPOLATION = cv2.INTER_LINEAR
# -----------------------------------

os.chdir(WORKDIR)
print("Working directory:", os.getcwd())

# Use exceptions for GDAL messages (optional)
try:
    gdal.UseExceptions()
except Exception:
    pass

# --- 1) Read image data (rasterio) ---
with rasterio.open(RAW_PATH) as src:
    raw_bands = src.read()  # shape (bands, h, w)
    raw_profile = src.profile
    raw_dtype = src.dtypes[0]
    print(f"Raw: {RAW_PATH}  bands={raw_bands.shape[0]}  shape={raw_bands.shape[1:]} dtype={raw_dtype}")

with rasterio.open(REF_PATH) as ref_src:
    ref_band1 = ref_src.read(1)  # single band used as reference for feature detection
    ref_transform = ref_src.transform
    ref_crs = ref_src.crs
    ref_h, ref_w = ref_band1.shape
    print(f"Ref: {REF_PATH}  shape={(ref_h, ref_w)}  crs={ref_crs}")

# --- 2) Prepare grayscale 8-bit images for feature detection ---
def to_8bit(img):
    """Normalize and convert to uint8 for feature detection (handles any dtype)."""
    img = img.astype("float32")
    mx = img.max()
    if mx == 0:
        return np.zeros_like(img, dtype=np.uint8)
    out = (img / mx * 255.0).astype(np.uint8)
    return out

# Use the first band of raw for feature detection (but preserve all bands for warping)
raw_band_for_feat = raw_bands[0]
raw_gray8 = to_8bit(raw_band_for_feat)
ref_gray8 = to_8bit(ref_band1)

# Save a small preview of the normalized images (optional)
cv2.imwrite(ALIGNED_PREVIEW, raw_gray8)
print("Saved preview for feature-detection:", ALIGNED_PREVIEW)

# --- 3) Feature detection & matching (ORB + KNN + Lowe) ---
# Increase ORB features for satellite imagery
orb = cv2.ORB_create(nfeatures=100000, edgeThreshold=15, patchSize=31)

kp1, des1 = orb.detectAndCompute(raw_gray8, None)
kp2, des2 = orb.detectAndCompute(ref_gray8, None)

if des1 is None or des2 is None:
    raise RuntimeError("No descriptors found in one of the images. Try different detector or check input images.")

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
matches = bf.knnMatch(des1, des2, k=2)

# Lowe's ratio test
good = []
for m_n in matches:
    if len(m_n) != 2:
        continue
    m, n = m_n
    if m.distance < RATIO_TEST * n.distance:
        good.append(m)

print("Good matches:", len(good))

if len(good) < MIN_MATCH_COUNT:
    raise RuntimeError(f"Not enough good matches ({len(good)}) to compute homography. Try relaxing ratio or using SIFT/AKAZE.")

# Draw debug matches (first 100)
dbg_img = cv2.drawMatches(raw_gray8, kp1, ref_gray8, kp2, good[:100], None,
                          flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imwrite(DEBUG_MATCHES, dbg_img)
print("Saved debug matches image:", DEBUG_MATCHES)

# --- 4) Compute homography (raw -> ref pixel coords) ---
src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
if H is None:
    raise RuntimeError("findHomography failed to produce a matrix.")

print("Homography matrix:\n", H)

# --- 5) Warp each band of the original to reference pixel grid ---
# raw_bands shape: (bands, raw_h, raw_w)
bands, raw_h, raw_w = raw_bands.shape
print(f"Warping {bands} bands from raw size {(raw_h, raw_w)} to ref size {(ref_h, ref_w)} ...")

warped_bands = np.zeros((bands, ref_h, ref_w), dtype=raw_bands.dtype)

for b in range(bands):
    band = raw_bands[b]
    # OpenCV expects single-channel 2D arrays; preserve dtype (e.g., uint16)
    # cv2.warpPerspective will output same dtype as input
    warped = cv2.warpPerspective(band, H, (ref_w, ref_h), flags=INTERPOLATION)
    warped_bands[b] = warped

print("Warping complete.")

# Optional: create a quick visual preview (8-bit) of the first warped band
warped_preview_8 = to_8bit(warped_bands[0])
cv2.imwrite("warped_preview_8bit.png", warped_preview_8)
print("Saved warped preview (8-bit): warped_preview_8bit.png")

# --- 6) Write multi-band GeoTIFF with reference geo-transform & CRS (rasterio) ---
out_profile = raw_profile.copy()
out_profile.update({
    "driver": "GTiff",
    "height": ref_h,
    "width": ref_w,
    "count": bands,
    "dtype": raw_bands.dtype,
    "transform": ref_transform,
    "crs": ref_crs,
    "compress": "lzw",
})

with rasterio.open(OUT_PATH, "w", **out_profile) as dst:
    dst.write(warped_bands)
    # if there is nodata in the original profile, also set it:
    if "nodata" in raw_profile and raw_profile["nodata"] is not None:
        dst.nodata = raw_profile["nodata"]

print(f"✅ Saved multi-band georeferenced image: {OUT_PATH}")


Working directory: D:\1. PROJECT\satellite_etl\satellite\test
Raw: sss.tif  bands=5  shape=(2309, 2018) dtype=uint16
Ref: oldimage.tif  shape=(2809, 2550)  crs=ESRI:102457
Saved preview for feature-detection: aligned_opencv_preview.tif
Good matches: 25
Saved debug matches image: debug_matches.png
Homography matrix:
 [[-5.10321727e+00  3.35473958e+00  1.88659914e+03]
 [-6.23889393e+00  4.20855694e+00  2.17433770e+03]
 [-2.81387303e-03  1.88157125e-03  1.00000000e+00]]
Warping 5 bands from raw size (2309, 2018) to ref size (2809, 2550) ...
Warping complete.
Saved warped preview (8-bit): warped_preview_8bit.png
✅ Saved multi-band georeferenced image: georeferenced.tif


In [4]:
import os
import cv2
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.transform import Affine
from osgeo import gdal

# ---------- User settings ----------
WORKDIR = r"D:\1. PROJECT\satellite_etl\satellite\test2"
RAW_PATH = "sss.tif"        # source to be aligned (may be multi-band, e.g., 16-bit)
REF_PATH = "oldimage.tif"   # reference with correct geo-transform & CRS
OUT_PATH = "georeferenced.tif"
DEBUG_MATCHES = "debug_matches.png"
ALIGNED_PREVIEW = "aligned_opencv_preview.tif"  # single-band preview
MIN_MATCH_COUNT = 4
RATIO_TEST = 0.75
# Choose interpolation: cv2.INTER_LINEAR (continuous) or cv2.INTER_NEAREST (categorical)
INTERPOLATION = cv2.INTER_LINEAR
# -----------------------------------

os.chdir(WORKDIR)
print("Working directory:", os.getcwd())

# Use exceptions for GDAL messages (optional)
try:
    gdal.UseExceptions()
except Exception:
    pass

# --- 1) Read image data (rasterio) ---
with rasterio.open(RAW_PATH) as src:
    raw_bands = src.read()  # shape (bands, h, w)
    raw_profile = src.profile
    raw_dtype = src.dtypes[0]
    print(f"Raw: {RAW_PATH}  bands={raw_bands.shape[0]}  shape={raw_bands.shape[1:]} dtype={raw_dtype}")

with rasterio.open(REF_PATH) as ref_src:
    ref_band1 = ref_src.read(1)  # single band used as reference for feature detection
    ref_transform = ref_src.transform
    ref_crs = ref_src.crs
    ref_h, ref_w = ref_band1.shape
    print(f"Ref: {REF_PATH}  shape={(ref_h, ref_w)}  crs={ref_crs}")

# --- 2) Prepare grayscale 8-bit images for feature detection ---
def to_8bit(img):
    """Normalize and convert to uint8 for feature detection (handles any dtype)."""
    img = img.astype("float32")
    mx = img.max()
    if mx == 0:
        return np.zeros_like(img, dtype=np.uint8)
    out = (img / mx * 255.0).astype(np.uint8)
    return out

# Use the first band of raw for feature detection (but preserve all bands for warping)
raw_band_for_feat = raw_bands[0]
raw_gray8 = to_8bit(raw_band_for_feat)
ref_gray8 = to_8bit(ref_band1)

# Save a small preview of the normalized images (optional)
cv2.imwrite(ALIGNED_PREVIEW, raw_gray8)
print("Saved preview for feature-detection:", ALIGNED_PREVIEW)

# --- 3) Feature detection & matching (ORB + KNN + Lowe) ---
# Option 1: AKAZE (fast, works on non-linear scale space, good for textured imagery)
# detector = cv2.AKAZE_create()

# Option 2: SIFT (robust, scale/rotation invariant, but slower)
detector = cv2.SIFT_create(nfeatures=10000)   


kp1, des1 = detector.detectAndCompute(raw_gray8, None)
kp2, des2 = detector.detectAndCompute(ref_gray8, None)

if des1 is None or des2 is None:
    raise RuntimeError("No descriptors found in one of the images. Try another detector.")

# Select matcher norm based on descriptor dtype
if des1.dtype == np.uint8:
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
else:
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

matches = bf.knnMatch(des1, des2, k=2)


# Lowe's ratio test
good = []
for m_n in matches:
    if len(m_n) != 2:
        continue
    m, n = m_n
    if m.distance < RATIO_TEST * n.distance:
        good.append(m)

print("Good matches:", len(good))


# Draw debug matches (first 100)
dbg_img = cv2.drawMatches(raw_gray8, kp1, ref_gray8, kp2, good[:100], None,
                          flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imwrite(DEBUG_MATCHES, dbg_img)
print("Saved debug matches image:", DEBUG_MATCHES)

# --- 4) Compute homography (raw -> ref pixel coords) ---
src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
if H is None:
    raise RuntimeError("findHomography failed to produce a matrix.")

print("Homography matrix:\n", H)

# --- 5) Warp each band of the original to reference pixel grid ---
# raw_bands shape: (bands, raw_h, raw_w)
bands, raw_h, raw_w = raw_bands.shape
print(f"Warping {bands} bands from raw size {(raw_h, raw_w)} to ref size {(ref_h, ref_w)} ...")

warped_bands = np.zeros((bands, ref_h, ref_w), dtype=raw_bands.dtype)

for b in range(bands):
    band = raw_bands[b]
    # OpenCV expects single-channel 2D arrays; preserve dtype (e.g., uint16)
    # cv2.warpPerspective will output same dtype as input
    warped = cv2.warpPerspective(band, H, (ref_w, ref_h), flags=INTERPOLATION)
    warped_bands[b] = warped

print("Warping complete.")

# Optional: create a quick visual preview (8-bit) of the first warped band
warped_preview_8 = to_8bit(warped_bands[0])
cv2.imwrite("warped_preview_8bit.png", warped_preview_8)
print("Saved warped preview (8-bit): warped_preview_8bit.png")

# --- 6) Write multi-band GeoTIFF with reference geo-transform & CRS (rasterio) ---
out_profile = raw_profile.copy()
out_profile.update({
    "driver": "GTiff",
    "height": ref_h,
    "width": ref_w,
    "count": bands,
    "dtype": raw_bands.dtype,
    "transform": ref_transform,
    "crs": ref_crs,
    "compress": "lzw",
})

with rasterio.open(OUT_PATH, "w", **out_profile) as dst:
    dst.write(warped_bands)
    # if there is nodata in the original profile, also set it:
    if "nodata" in raw_profile and raw_profile["nodata"] is not None:
        dst.nodata = raw_profile["nodata"]

print(f"✅ Saved multi-band georeferenced image: {OUT_PATH}")


Working directory: D:\1. PROJECT\satellite_etl\satellite\test2
Raw: sss.tif  bands=5  shape=(2309, 2018) dtype=uint16
Ref: oldimage.tif  shape=(2809, 2550)  crs=ESRI:102457
Saved preview for feature-detection: aligned_opencv_preview.tif
Good matches: 31
Saved debug matches image: debug_matches.png
Homography matrix:
 [[-1.95431806e+00  6.94694982e-01  2.33320680e+03]
 [-1.61250824e+00  5.72370270e-01  1.92594337e+03]
 [-8.37444740e-04  2.97508979e-04  1.00000000e+00]]
Warping 5 bands from raw size (2309, 2018) to ref size (2809, 2550) ...
Warping complete.
Saved warped preview (8-bit): warped_preview_8bit.png
✅ Saved multi-band georeferenced image: georeferenced.tif
